In [1]:
!pip install hvplot
!pip install jupyter_bokeh


In [2]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Para crear el dashboard
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [3]:
# Cargar el dataframe
df = pd.read_csv('./email_phishing_data.csv')
email = df.copy()
df.head()

,num_words,num_unique_words,num_stopwords,num_links,num_unique_domains,num_email_addresses,num_spelling_errors,num_urgent_keywords,label
0,140,94,52,0,0,0,0,0,0
1,5,5,1,0,0,0,0,0,0
2,34,32,15,0,0,0,0,0,0
3,6,6,2,0,0,0,0,0,0
4,9,9,2,0,0,0,0,0,0


In [4]:
df.describe()

,num_words,num_unique_words,num_stopwords,num_links,num_unique_domains,num_email_addresses,num_spelling_errors,num_urgent_keywords,label
count,5.248460e+05,524846.000000,524846.000000,524846.000000,524846.000000,524846.000000,524846.000000,524846.000000,524846.000000
mean,2.762280e+02,123.012167,80.045465,0.895588,0.347767,2.114897,24.694731,0.245301,0.013240
std,3.335457e+03,201.626478,1023.330380,5.603001,1.774209,13.592682,311.312358,0.559320,0.114301
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.700000e+01,38.000000,12.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
50%,1.200000e+02,79.000000,34.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000
75%,2.690000e+02,145.000000,79.000000,0.000000,0.000000,1.000000,22.000000,0.000000,0.000000
max,2.339682e+06,51251.000000,720411.000000,824.000000,524.000000,1150.000000,190104.000000,7.000000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524846 entries, 0 to 524845
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   num_words            524846 non-null  int64
 1   num_unique_words     524846 non-null  int64
 2   num_stopwords        524846 non-null  int64
 3   num_links            524846 non-null  int64
 4   num_unique_domains   524846 non-null  int64
 5   num_email_addresses  524846 non-null  int64
 6   num_spelling_errors  524846 non-null  int64
 7   num_urgent_keywords  524846 non-null  int64
 8   label                524846 non-null  int64
dtypes: int64(9)
memory usage: 36.0 MB


In [6]:
# Limpieza de datos
df.isnull().sum()

num_words              0
num_unique_words       0
num_stopwords          0
num_links              0
num_unique_domains     0
num_email_addresses    0
num_spelling_errors    0
num_urgent_keywords    0
label                  0
dtype: int64

In [7]:
# Diccionario para traducir nombres de columnas al español
column_translation = {
    'num_words': 'Número de Palabras',
    'num_unique_words': 'Palabras Únicas',
    'num_stopwords': 'Palabras Vacías',
    'num_links': 'Número de Enlaces',
    'num_unique_domains': 'Dominios Únicos',
    'num_email_addresses': 'Direcciones de Email',
    'num_spelling_errors': 'Errores Ortográficos',
    'num_urgent_keywords': 'Palabras Urgentes',
    'label': 'Etiqueta'
}

# Crear copia del dataframe con nombres traducidos
df_spanish = df.copy()
df_spanish = df_spanish.rename(columns=column_translation)

# Cambiar etiquetas de clasificación
df_spanish['Etiqueta'] = df_spanish['Etiqueta'].map({0: 'Seguro', 1: 'Phishing'})

# Agrupar por 'Etiqueta' y calcular la media de las características
df_grouped = df_spanish.groupby('Etiqueta').mean()

# Reiniciar el índice para que 'Etiqueta' sea una columna normal
df_grouped_reset = df_grouped.reset_index()

# Renombrar las columnas para que sean más descriptivas
df_melted = df_grouped_reset.melt(id_vars='Etiqueta', var_name='caracteristica', value_name='valor_medio')

print("Dataset procesado con etiquetas en español:")
display(df_melted.head(10))

Dataset procesado con etiquetas en español:


,Etiqueta,caracteristica,valor_medio
0,Phishing,Número de Palabras,246.150957
1,Seguro,Número de Palabras,276.631566
2,Phishing,Palabras Únicas,128.300907
3,Seguro,Palabras Únicas,122.941205
4,Phishing,Palabras Vacías,75.711901
5,Seguro,Palabras Vacías,80.103611
6,Phishing,Número de Enlaces,0.423658
7,Seguro,Número de Enlaces,0.901921
8,Phishing,Dominios Únicos,0.309109
9,Seguro,Dominios Únicos,0.348285


In [8]:
# Crear gráficos de barras para cada característica
bar_plots = {}
for feature in df_melted['caracteristica'].unique():
    plot_data = df_melted[df_melted['caracteristica'] == feature]
    bar_plots[feature] = plot_data.hvplot.bar(
        x='Etiqueta',
        y='valor_medio',
        title=f'Promedio de {feature} por Categoría',
        xlabel='Categoría',
        ylabel=f'Promedio de {feature}',
        height=400,
        width=600,
        color=['#2E86AB', '#A23B72'],
        legend=False,
        tools=['hover']
    )

# Crear gráficos adicionales para datos individuales
scatter_plots = {}
for col in df_spanish.columns[:-1]:  # Excluir la columna 'Etiqueta'
    scatter_plots[col] = df_spanish.hvplot.scatter(
        x=col,
        y='Etiqueta',
        by='Etiqueta',
        title=f'Distribución de {col}',
        xlabel=col,
        ylabel='Categoría',
        height=400,
        width=600,
        alpha=0.6,
        size=50
    )

print(f"Creados {len(bar_plots)} gráficos de barras y {len(scatter_plots)} gráficos de dispersión")

Creados 8 gráficos de barras y 8 gráficos de dispersión


In [9]:
# Función para filtrar datos
def filtrar_datos(categoria, caracteristica_min=None, caracteristica_max=None, caracteristica_col=None):
    """Filtrar el dataframe según los criterios seleccionados"""
    df_filtrado = df_spanish.copy()
    
    # Filtrar por categoría
    if categoria != 'Todos':
        df_filtrado = df_filtrado[df_filtrado['Etiqueta'] == categoria]
    
    # Filtrar por rango de características si se especifica
    if caracteristica_col and caracteristica_min is not None and caracteristica_max is not None:
        df_filtrado = df_filtrado[
            (df_filtrado[caracteristica_col] >= caracteristica_min) & 
            (df_filtrado[caracteristica_col] <= caracteristica_max)
        ]
    
    return df_filtrado

# Función para crear tabla de datos filtrados
def crear_tabla_datos(df_filtrado):
    """Crear tabla interactiva con los datos filtrados"""
    if len(df_filtrado) > 0:
        return pn.widgets.Tabulator(
            df_filtrado.head(100),  # Mostrar primeros 100 registros
            title=f"Datos Filtrados ({len(df_filtrado)} registros total)",
            height=400,
            width=900,
            pagination='remote',
            page_size=20
        )
    else:
        return pn.pane.HTML("<p>No hay datos que cumplan con los filtros seleccionados</p>")

print("Funciones de filtrado creadas exitosamente")

Funciones de filtrado creadas exitosamente


In [10]:
# Instalar plotly si no está disponible
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
except ImportError:
    !pip install plotly
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

# Función alternativa para crear pie charts interactivos con Plotly
def crear_pie_charts_plotly():
    pie_charts = {}
    
    # Calcular porcentajes para cada categoría
    for categoria in ['Phishing', 'Seguro']:
        df_categoria = df_spanish[df_spanish['Etiqueta'] == categoria]
        
        # Calcular la suma total de cada característica para esta categoría
        totales = {}
        for col in df_spanish.columns[:-1]:  # Excluir columna 'Etiqueta'
            totales[col] = df_categoria[col].sum()
        
        # Crear DataFrame para el pie chart
        df_pie = pd.DataFrame(list(totales.items()), columns=['Característica', 'Total'])
        df_pie['Porcentaje'] = (df_pie['Total'] / df_pie['Total'].sum() * 100).round(1)
        
        # Colores personalizados
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8', '#F7DC6F']
        
        # Crear pie chart interactivo con Plotly
        fig = go.Figure(data=[go.Pie(
            labels=df_pie['Característica'],
            values=df_pie['Total'],
            hole=0.3,  # Hacer un donut chart para mejor visualización
            marker_colors=colors,
            textinfo='label+percent',
            textposition='outside',
            textfont_size=12,
            pull=[0.05] * len(df_pie)  # Separar ligeramente las secciones
        )])
        
        # Configurar el layout
        fig.update_layout(
            title={
                'text': f'Distribución de Características - {categoria}',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 16, 'family': 'Arial Black'}
            },
            showlegend=True,
            legend=dict(
                orientation="v",
                yanchor="middle",
                y=0.5,
                xanchor="left",
                x=1.05
            ),
            width=600,
            height=500,
            margin=dict(l=50, r=150, t=80, b=50)
        )
        
        # Agregar interactividad con hover personalizado
        fig.update_traces(
            hovertemplate='<b>%{label}</b><br>' +
                         'Total: %{value:,.0f}<br>' +
                         'Porcentaje: %{percent}<br>' +
                         '<extra></extra>'
        )
        
        # Convertir a panel pane
        pie_charts[categoria] = pn.pane.Plotly(fig, width=600, height=500)
    
    return pie_charts

print("✅ Función de pie charts interactivos con Plotly creada!")

✅ Función de pie charts interactivos con Plotly creada!


In [11]:
# Crear pie charts comparativos para cada categoría
def crear_pie_charts_comparativos():
    pie_charts = {}
    
    # Calcular porcentajes para cada categoría
    for categoria in ['Phishing', 'Seguro']:
        df_categoria = df_spanish[df_spanish['Etiqueta'] == categoria]
        
        # Calcular la suma total de cada característica para esta categoría
        totales = {}
        for col in df_spanish.columns[:-1]:  # Excluir columna 'Etiqueta'
            totales[col] = df_categoria[col].sum()
        
        # Crear DataFrame para el pie chart
        df_pie = pd.DataFrame(list(totales.items()), columns=['Característica', 'Total'])
        df_pie['Porcentaje'] = (df_pie['Total'] / df_pie['Total'].sum() * 100).round(1)
        
        # Definir colores para cada categoría
        colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8', '#F7DC6F']
        
        # Crear pie chart usando matplotlib con mejor legibilidad
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Crear el pie chart con explode para separar las secciones
        explode = [0.05] * len(df_pie)  # Separar todas las secciones ligeramente
        wedges, texts, autotexts = ax.pie(
            df_pie['Total'], 
            labels=None,  # No mostrar etiquetas directamente en el gráfico
            autopct='%1.1f%%',
            colors=colors,
            startangle=90,
            explode=explode,
            textprops={'fontsize': 10, 'fontweight': 'bold'}
        )
        
        # Configurar el título
        ax.set_title(f'Distribución de Características - {categoria}', 
                    fontsize=16, fontweight='bold', pad=20)
        
        # Crear leyenda externa para mejor legibilidad
        ax.legend(wedges, df_pie['Característica'],
                 title="Características",
                 loc="center left",
                 bbox_to_anchor=(1, 0, 0.5, 1),
                 fontsize=10)
        
        # Mejorar la legibilidad de los porcentajes
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
            autotext.set_fontsize(9)
        
        # Ajustar el layout para que no se corte la leyenda
        plt.tight_layout()
        
        # Convertir a panel pane con navegación interactiva habilitada
        pie_charts[categoria] = pn.pane.Matplotlib(
            fig, 
            tight=True, 
            width=600, 
            height=500,
            sizing_mode='stretch_width'
        )
        plt.close(fig)  # Cerrar la figura para evitar memory leaks
    
    return pie_charts

# Crear los pie charts interactivos (usando Plotly para zoom y mejor interactividad)
pie_charts = crear_pie_charts_plotly()

print("✅ Pie charts interactivos con zoom creados exitosamente!")



✅ Pie charts interactivos con zoom creados exitosamente!


In [12]:
# Función para mostrar pie charts comparativos
def mostrar_pie_charts_comparativos():
    # Retornar los pie charts ya creados en la función de abajo
    return pie_charts

# Crear widgets de control simplificados
categoria_select = pn.widgets.Select(
    name='Filtrar por Categoría',
    options=['Todos', 'Seguro', 'Phishing'],
    value='Todos'
)

caracteristica_select = pn.widgets.Select(
    name='Seleccionar Característica',
    options=list(bar_plots.keys()),
    value=list(bar_plots.keys())[0]
)

# Usar pie charts en lugar de histogramas
pie_charts_display = mostrar_pie_charts_comparativos()

# Función para crear métricas de resumen
def crear_metricas(categoria):
    if categoria == 'Todos':
        total = len(df_spanish)
        seguro_count = len(df_spanish[df_spanish['Etiqueta'] == 'Seguro'])
        phishing_count = len(df_spanish[df_spanish['Etiqueta'] == 'Phishing'])
        
        metricas_html = f"""
        <div style="display: flex; gap: 20px; margin: 20px 0;">
            <div style="background: linear-gradient(135deg, #2E86AB, #4CAF50); color: white; padding: 20px; border-radius: 10px; text-align: center; flex: 1;">
                <h2 style="margin: 0; font-size: 2em;">{total:,}</h2>
                <p style="margin: 5px 0 0 0;">Total de Emails</p>
            </div>
            <div style="background: linear-gradient(135deg, #4CAF50, #8BC34A); color: white; padding: 20px; border-radius: 10px; text-align: center; flex: 1;">
                <h2 style="margin: 0; font-size: 2em;">{seguro_count:,}</h2>
                <p style="margin: 5px 0 0 0;">Emails Seguros</p>
                <small>({seguro_count/total*100:.1f}%)</small>
            </div>
            <div style="background: linear-gradient(135deg, #A23B72, #F44336); color: white; padding: 20px; border-radius: 10px; text-align: center; flex: 1;">
                <h2 style="margin: 0; font-size: 2em;">{phishing_count:,}</h2>
                <p style="margin: 5px 0 0 0;">Emails Phishing</p>
                <small>({phishing_count/total*100:.1f}%)</small>
            </div>
        </div>
        """
    else:
        df_filtrado = df_spanish[df_spanish['Etiqueta'] == categoria]
        total = len(df_filtrado)
        metricas_html = f"""
        <div style="background: linear-gradient(135deg, #2E86AB, #A23B72); color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 20px 0;">
            <h2 style="margin: 0; font-size: 2em;">{total:,}</h2>
            <p style="margin: 5px 0 0 0;">Emails - {categoria}</p>
        </div>
        """
    
    return pn.pane.HTML(metricas_html)

# Función para crear tabla de datos filtrados
def crear_tabla(categoria):
    if categoria == 'Todos':
        df_filtrado = df_spanish
    else:
        df_filtrado = df_spanish[df_spanish['Etiqueta'] == categoria]
    
    titulo = pn.pane.Markdown(f"### Datos Filtrados - {categoria} ({len(df_filtrado)} registros total)")
    tabla = pn.widgets.Tabulator(
        df_filtrado.head(50),  # Mostrar primeros 50 registros
        pagination='remote',
        page_size=10,
        sizing_mode='stretch_width',
        layout='fit_columns',
        show_index=False
    )
    
    return pn.Column(titulo, tabla)

# Crear componentes interactivos
grafico_barras = pn.bind(lambda caracteristica: bar_plots[caracteristica], caracteristica_select)
histograma_comparativo = pn.bind(lambda caracteristica: pn.Row(pie_charts['Phishing'], pie_charts['Seguro']), caracteristica_select)
metricas_interactivas = pn.bind(crear_metricas, categoria_select)
tabla_interactiva = pn.bind(crear_tabla, categoria_select)

# Crear el dashboard usando FastListTemplate
template = pn.template.FastListTemplate(
    title='🛡️ Dashboard de Análisis de Phishing en Emails',
    sidebar=[
        pn.pane.Markdown("""
        ## 🎛️ Controles del Dashboard
        
        Selecciona los filtros para explorar los datos de emails.
        """),
        
        categoria_select,
        caracteristica_select,
        
        pn.pane.Markdown("""
        ### 📋 Información
        
        **Categorías:**
        - 🟢 **Seguro**: Emails legítimos
        - 🔴 **Phishing**: Emails maliciosos
        
        **Características analizadas:**
        - Número de palabras y palabras únicas
        - Enlaces y dominios
        - Errores ortográficos
        - Palabras urgentes
        """)
    ],
    
    main=[
        pn.pane.Markdown("## 📊 Métricas Principales"),
        metricas_interactivas,
        
        pn.Row(
            pn.Column(
                pn.pane.Markdown("### 📈 Promedio por Categoría"),
                grafico_barras,
                width=600
            ),
            pn.Column(
                pn.pane.Markdown("### 📊 Comparativa de Pie Charts"),
                histograma_comparativo,
                width=600
            )
        ),
        
        pn.pane.Markdown("## 📋 Datos Detallados"),
        tabla_interactiva
    ],
    
    accent_base_color='#2E86AB',
    header_background='#1f2937',
    main_max_width='1400px'
)

# Mostrar el dashboard
template.show()

Launching server at http://localhost:53950
